Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

In [1]:
import pandas as pd                     
import numpy as np
from scipy.stats import shapiro         # тест Шапиро-Уилка
from scipy.stats import normaltest      # тест Д'Агостино
from scipy.stats import ttest_ind       # Независимый Т-тест
from scipy.stats import f_oneway        # ANOVA тест


In [2]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [3]:
print(len(petersburg), len(magadan))    # определяем длины массивов

7 8


In [4]:
petersburg.append(np.nan)               # добавляем еще одно значение в короткий массив

In [5]:
mussels = pd.DataFrame({'Petersburg': petersburg, 'Magadan': magadan})  # формируем DataFrame
display(mussels)

,Petersburg,Magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


In [6]:
values = {'Petersburg': mussels['Petersburg'].mean()}  # заполняем пропущенное значение средним
mussels = mussels.fillna(values)
display(mussels)

,Petersburg,Magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


### Проверка данных на нормальность

In [7]:
H0 = 'Данные распределены нормально'            # нулевая гипотеза
Ha = 'Данные не распределены нормально'         # альтернативная гипотеза
alpha = 0.01                                   # устанавливаем уровень значимости

_, p = shapiro(mussels['Petersburg'])           # тест Шапиро - Уилка (подходит для небольших выборок данных)
print('Petersburg: p=%.3f' % p)             
if p > alpha:
	print(H0)
else:
	print(Ha)

_, p = shapiro(mussels['Magadan'])                
print('Magadan: p=%.3f' % p)             
if p > alpha:
	print(H0)
else:
	print(Ha)

Petersburg: p=0.123
Данные распределены нормально
Magadan: p=0.036
Данные распределены нормально


В том случае, если мы выбираем достаточно высокий уровень точности, мы получаем, что данные обоих признаков распределены нормально.
Возможно, было бы корректнее выбрать средний уровень значимости alpha = 0.05. В этом случае распределение для Магадана не было бы нормальным, и следовало бы использовать непараметрические тесты. Однако, в рамках данного модуля мы изучаем именно нормальное распределение и параметрические тесты. Поэтому выставляем соответствующее значение alpha

##### Тест на корреляцию

##### Для определения корреляции мы используем метод Пирсона (он бы использовался и по умолчанию). Этот метод подходит для количественных непрерывных значений, распределенных нормально


In [8]:
mussels.corr(method='pearson') 

,Petersburg,Magadan
Petersburg,1.000000,0.244558
Magadan,0.244558,1.000000


##### результат - 0.244, что говорит о слабой корреляции

#### Независимый Т-тест

Т-тест (t-критерий) используется при сравнении средних значений ровно двух групп одной совокупности (в нашем случае Петербунг и Магадан). 
Зависимая переменная (размер мидий) является количественной и подчиняется нормальному закону распределения.
Следовательно, мы используем независимый T-тест.

In [9]:
H0 = 'Нет значимой разницы между размерами мидий из Петербурга и Магадана.'
Ha = 'Есть значимая разница между между размерами мидий из Петербурга и Магадана.'

def t_test(df):
    print('\n'+ "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(mussels['Petersburg'], mussels['Magadan'], equal_var=True)

    p = round(test_results[1],4)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(mussels)


*** Результаты независимого T-теста ***
0.0027 <= 0.01. Мы отвергаем нулевую гипотезу. Есть значимая разница между между размерами мидий из Петербурга и Магадана.


По результатам независимого Т-теста мы выяснили, что значение p-value (вероятность обнаружения наблюдаемых результатов, когда нулевая гипотеза верна) составляет 0.0027, что значительно меньше, чем уровень значимости alpha (фиксированная вероятность ошибочного отклонения истинной нулевой гипотезы, вероятность ошибки I рода)
P-value нашего теста ниже уровня значимости, это означает, что результаты статистически значимы и согласуются с альтернативной гипотезой, которая говорит о том, что есть значимая разница между размерами мидий из Петербурга и Магадана.

##### ANOVA test

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

In [10]:
H0 = 'Нет значимой разницы между средним размером раковины мидий из Петербурга и Магадана.'
Ha = 'Есть значимая разница между средним размером раковины мидий из Петербурга и Магадана.'

In [11]:
_, p = f_oneway(mussels['Petersburg'], mussels['Magadan'])

if p>alpha:
  print(f"{'p=%.4f' % p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{'p=%.4f' % p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

p=0.0027 <= 0.01. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий из Петербурга и Магадана.


Наша нулевая гипотеза о том, что средние значения размеров раковин мидий из Петербурга и Магадана равны, оказалась неверной.

##### Ответ на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя

In [12]:
p = mussels['Petersburg'].mean()
m = mussels['Magadan'].mean()
diff = round(p - m, 4)
display(f'Разница в среднем размере мидии в зависимости от города-производителя составляет {diff} ')

'Разница в среднем размере мидии в зависимости от города-производителя составляет 0.0254 '